In [1]:
#!gdown 1fYtaFcGKSL8ykJFbewAFsFf9C4aW0DuV
#!gdown 1VILdkCRWsDTrN9DPeMLh8jlibAoBLzy-
#!gdown 197pDAI8KVsiDAA1xaPbitZpmvzh9CDqT
#!gdown 1XqgXAtxWUdnBJnvQTtEr8JpBkVi3JuSC
#!gdown 1_DVttDvwxCm_QGDxU2tbaaclWKd1VCau

In [2]:
#%mkdir -p /data
#%mv /content/*.hdf5 /data/
#%mv /content/*.npy /data/

In [3]:
#from google.colab import drive
#drive.mount('/content/gdrive/')
#import sys
#sys.path.append('/content/gdrive/MyDrive/Universidad/VIU/TFM/notebooks')

In [4]:
%%capture
%pip install torch torchvision h5py xarray matplotlib netcdf4

In [5]:
import random
import time
import os
import datetime
import itertools
import math
import sys

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.utils.data as data
from torch.utils.data import DataLoader
import numpy as np
import h5py
from PIL import Image
import matplotlib.pyplot as plt
import xarray as xr

sys.path.append('../common')
import common
from ConvLSTMmodels import ConvLSTM, EncoderRNN, VideoDataset, ConvLSTMEncoderDecoder

In [6]:
# check tensorflow version
print("pytorch version:", torch.__version__)
# check available gpu
if torch.cuda.is_available():
    gpus =  torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Available GPUs: {gpus}")
else:
    print("GPUs not available. Only CPU.")

pytorch version: 2.3.1+cu121
GPUs not available. Only CPU.


In [7]:
cwd = os.getcwd()
pardir = os.path.dirname(os.path.dirname(cwd))
data_folder = os.path.join(pardir,'data')
data_path = os.path.join(data_folder,'video_prediction_dataset.hdf5')
model_name = 'ConvLSTM'
output_folder = os.path.join(cwd,"save", model_name)
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)
print("data_folder:", data_folder)
print("data_path:", data_path)
print("output_folder:", output_folder)

data_folder: /home/javgat/Repos/Otros/presente/data
data_path: /home/javgat/Repos/Otros/presente/data/video_prediction_dataset.hdf5
output_folder: /home/javgat/Repos/Otros/presente/codes/ConvLSTM/save/ConvLSTM


In [8]:
def get_all(name):
    # show structure of the hdf5 data
    if name!=None:
        print(forecast_dataset[name])

# generate handler for the hdf5 data
with h5py.File(data_path, 'r') as forecast_dataset:
    forecast_dataset.visit(get_all)

<HDF5 group "/test" (4 members)>
<HDF5 dataset "images_log": shape (4467, 16, 64, 64, 3), type "|u1">
<HDF5 dataset "images_pred": shape (4467, 15, 64, 64, 3), type "|u1">
<HDF5 dataset "pv_log": shape (4467, 16), type "<f4">
<HDF5 dataset "pv_pred": shape (4467, 15), type "<f4">
<HDF5 group "/trainval" (4 members)>
<HDF5 dataset "images_log": shape (53336, 16, 64, 64, 3), type "|u1">
<HDF5 dataset "images_pred": shape (53336, 15, 64, 64, 3), type "|u1">
<HDF5 dataset "pv_log": shape (53336, 16), type "<f4">
<HDF5 dataset "pv_pred": shape (53336, 15), type "<f4">


In [9]:
with h5py.File(data_path, 'r') as fds:
    group_names = list(fds.keys())
    print(group_names)

dss = {}
for gname in group_names:
    dss[gname] = xr.open_dataset(data_path, group=gname)

['test', 'trainval']


In [10]:
print(dss['test']['images_log'])

<xarray.DataArray 'images_log' (phony_dim_0: 4467, phony_dim_1: 16,
                                phony_dim_2: 64, phony_dim_3: 64, phony_dim_4: 3)> Size: 878MB
[878247936 values with dtype=uint8]
Dimensions without coordinates: phony_dim_0, phony_dim_1, phony_dim_2,
                                phony_dim_3, phony_dim_4


In [11]:
dss['test']['images_log'].shape

(4467, 16, 64, 64, 3)

In [12]:
times_curr_train = np.load(os.path.join(data_folder,"times_curr_trainval.npy"),allow_pickle=True)
times_curr_test = np.load(os.path.join(data_folder,"times_curr_test.npy"),allow_pickle=True)

print('-'*50)
print("times_curr_train.shape:", times_curr_train.shape)
print("images_log_train.shape:", dss['trainval']['images_log'].shape)
print("images_pred_train.shape:", dss['trainval']['images_pred'].shape)
print("times_curr_test.shape:", times_curr_test.shape)
print("images_log_test.shape:", dss['test']['images_log'].shape)
print("images_pred_test.shape:", dss['test']['images_pred'].shape)
print('-'*50)

# get the input dimension for constructing the model
num_log_frame = dss['trainval']['images_log'].shape[1]
img_side_len = dss['trainval']['images_log'].shape[2]
num_color_channel = dss['trainval']['images_log'].shape[4]
num_pred_frame = dss['trainval']['images_pred'].shape[1]
image_log_dim = [num_log_frame,img_side_len,img_side_len,num_color_channel]
image_pred_dim = [num_pred_frame,img_side_len,img_side_len,num_color_channel]

print("image side length:", img_side_len)
print("number of log frames:", num_log_frame)
print("number of pred frames:", num_pred_frame)
print("number of color channels:", num_color_channel)
print("context(log) image dimension:", image_log_dim)
print("future(pred) image dimension:", image_pred_dim)

--------------------------------------------------
times_curr_train.shape: (53336,)
images_log_train.shape: (53336, 16, 64, 64, 3)
images_pred_train.shape: (53336, 15, 64, 64, 3)
times_curr_test.shape: (4467,)
images_log_test.shape: (4467, 16, 64, 64, 3)
images_pred_test.shape: (4467, 15, 64, 64, 3)
--------------------------------------------------
image side length: 64
number of log frames: 16
number of pred frames: 15
number of color channels: 3
context(log) image dimension: [16, 64, 64, 3]
future(pred) image dimension: [15, 64, 64, 3]


### dataloader

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
batch_size_train = 16
batch_size_test = 64
if device.type == "cpu":
  batch_size_train = 4
  batch_size_test = 4
nepochs = 50
print('nepochs:',nepochs)
print_every = 1
eval_every = 5

cpu
nepochs: 50


In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.float()),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Example normalization
])

In [15]:
# Create Dataset and DataLoader
train_dataset = VideoDataset(dss['trainval']['images_log'], dss['trainval']['images_pred'], transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)

test_dataset = VideoDataset(dss['test']['images_log'], dss['test']['images_pred'], transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size_test, shuffle=False)

### Training and Validate the Model

#### Define train & evaluate

In [16]:
def train_on_minibatch(input_tensor, target_tensor, model, optimizer, criterion, teacher_forcing_ratio, accumulation_steps, batch_idx):
    target_length = target_tensor.shape[1]
    pred = model(input_tensor, target_length)
    pred = pred
    print(pred.shape)
    print(target_tensor.shape)
    loss = criterion(pred, target_tensor)
    loss = loss / accumulation_steps
    loss.backward(retain_graph=True)
    
    # Update optimizer and zero grad only when accumulation step is reached
    if (batch_idx + 1) % accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
    
    return loss.item() / target_tensor.size(1)

In [17]:
def evaluate(model, loader, device, times_curr_test, img_side_len, num_color_channel):
    model.eval()
    def compute_metrics(prediction, target):
        mse = np.mean((prediction - target) ** 2, axis=1).sum()
        mae = np.mean(np.abs(prediction - target), axis=1).sum()
        return mse, mae

    print("Validation start...")
    total_mse, total_mae = 0, 0
    start_time = time.time()
    predictions = []
    indices = []

    num_val_samples = len(times_curr_test)
    with torch.no_grad():
        for idx, (inseq, outseq) in enumerate(loader):
            batch_indices = idx
            input_tensor = inseq.to(device)
            target_tensor = outseq.to(device)
            target_length = target_tensor.shape[1]
            pred = model(input_tensor, target_length)
            pred = pred[0]

            # Compute metrics for the current batch
            mse_batch, mae_batch = compute_metrics(pred, target_tensor)
            total_mse += mse_batch
            total_mae += mae_batch

            indices.extend(batch_indices)

    # Compute overall metrics
    predictions = np.concatenate(predictions, axis=0)
    mse_per_frame = total_mse / num_val_samples
    mae_per_frame = total_mae / num_val_samples
    mse_per_pixel = mse_per_frame / (img_side_len * img_side_len * num_color_channel)
    mae_per_pixel = mae_per_frame / (img_side_len * img_side_len * num_color_channel)

    print(f'Eval MSE per frame: {mse_per_frame:.4f}')
    print(f'Eval MAE per frame: {mae_per_frame:.4f}')
    print(f'Eval MSE per pixel: {mse_per_pixel:.6f}')
    print(f'Eval MAE per pixel: {mae_per_pixel:.6f}')
    print(f'Time: {time.time() - start_time:.2f}s')

    return mse_per_frame, mae_per_frame, mse_per_pixel, mae_per_pixel, predictions, indices


In [18]:
def trainIters(model, nepochs, train_loader, test_loader, device, print_every=10, eval_every=10, name='', accumulation_steps=4):
  train_losses = []
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  scheduler_enc = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.1)
  criterion = nn.MSELoss()
  lrs = []

  for epoch in range(nepochs):
    model.train()
    t0 = time.time()
    loss_epoch = 0
    teacher_forcing_ratio = max(0, 1 - epoch * 0.03)

    for batch_idx, (input_seq, output_seq) in enumerate(train_loader):
      print(batch_idx)
      input_tensor = input_seq.to(device)
      target_tensor = output_seq.to(device)
      loss = train_on_minibatch(input_tensor, target_tensor, model, optimizer, criterion, teacher_forcing_ratio, accumulation_steps, batch_idx)
      loss_epoch += loss

    # Ensure any remaining gradients are applied
    if (batch_idx + 1) % accumulation_steps != 0:
      optimizer.step()
      optimizer.zero_grad()

    train_losses.append(loss_epoch)

    if (epoch + 1) % print_every == 0:
      print('-' * 50)
      print(f'Epoch {epoch + 1} | Loss: {loss_epoch:.4f} | Time: {time.time() - t0:.2f}s')
      print("Saving model...")
      torch.save(model.state_dict(), f'save/{name}/encoder.pth')

    if (epoch + 1) % eval_every == 0:
      mse_per_frame, _, mse_per_pixel, _, _, _ = evaluate(model, test_loader)
      scheduler_enc.step(mse_per_frame)

    lrs.append(scheduler_enc.get_last_lr())

  return train_losses, lrs

#### Actually train

In [19]:
input_dim = 3
hidden_dims = [16, 16] #[128,128,64]
num_layers = 2 #3
kernel_size = (3, 3)
batch_first = True
#convlstm = ConvLSTM(input_dim, hidden_dims, kernel_size, num_layers, batch_first)
#model = EncoderRNN(convcell, device, nc=3)
model = ConvLSTMEncoderDecoder(input_dim, hidden_dims, kernel_size, num_layers, batch_first)
model = model.to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('model ', count_parameters(model) )

model  66547


In [20]:
train_loss, learning_rates = trainIters(model, nepochs, train_loader, test_loader, device, print_every, eval_every, model_name)

0
15
torch.Size([4, 1, 16, 64, 64])
0
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
1
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
2
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
3
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
4
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
5
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
6
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
7
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
8
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
9
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
10
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
11
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
12
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
13
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
14
torch.Size([4, 1, 16, 64, 64])
torch.Size([4, 1, 3, 64, 64])
torch.Size([4,

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [64, 32, 3, 3]] is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

        ### Save Predicted Images from Validation Set

In [ ]:
model.load_state_dict(torch.load('save/{0}/encoder.pth'.format(model_name)))
model.eval()
mse_per_frame, mae_per_frame, mse_per_pixel, mae_per_pixel, predictions, indices = evaluate(model,test_loader)

In [ ]:
print(predictions.shape)
predictions = predictions.transpose((0,1,3,4,2))
print(predictions.shape)

In [ ]:
np.save('save/{0}/predicted_images.npy'.format(model_name), predictions)

### Visualize Some Sample Predictions

In [ ]:
random.seed(0)
select_num_samples = 30
select_idx = random.sample(np.arange(len(predictions)).tolist(),select_num_samples)

In [ ]:
for i in range(select_num_samples):
    print("-"*50,"sample ",str(i+1), "-"*50)
    f, ax = plt.subplots(2,8)
    f.set_size_inches(24,6)
    ax[0,0].imshow(images_log_test[select_idx[i]][0][:,:,::-1])
    ax[0,0].set_title(times_curr_test[select_idx[i]]-datetime.timedelta(minutes=15))
    ax[0,1].imshow(images_log_test[select_idx[i]][2][:,:,::-1])
    ax[0,1].set_title(times_curr_test[select_idx[i]]-datetime.timedelta(minutes=11))
    ax[0,2].imshow(images_log_test[select_idx[i]][4][:,:,::-1])
    ax[0,2].set_title(times_curr_test[select_idx[i]]-datetime.timedelta(minutes=7))
    ax[0,3].imshow(images_log_test[select_idx[i]][7][:,:,::-1])
    ax[0,3].set_title(times_curr_test[select_idx[i]]-datetime.timedelta(minutes=1))
    ax[0,4].imshow(images_pred_test[select_idx[i]][0][:,:,::-1])
    ax[0,4].set_title(times_curr_test[select_idx[i]]+datetime.timedelta(minutes=1))
    ax[0,5].imshow(images_pred_test[select_idx[i]][2][:,:,::-1])
    ax[0,5].set_title(times_curr_test[select_idx[i]]+datetime.timedelta(minutes=5))
    ax[0,6].imshow(images_pred_test[select_idx[i]][4][:,:,::-1])
    ax[0,6].set_title(times_curr_test[select_idx[i]]+datetime.timedelta(minutes=9))
    ax[0,7].imshow(images_pred_test[select_idx[i]][7][:,:,::-1])
    ax[0,7].set_title(times_curr_test[select_idx[i]]+datetime.timedelta(minutes=15))

    ax[1,4].imshow(predictions[select_idx[i]][0][:,:,::-1])
    ax[1,5].imshow(predictions[select_idx[i]][2][:,:,::-1])
    ax[1,6].imshow(predictions[select_idx[i]][4][:,:,::-1])
    ax[1,7].imshow(predictions[select_idx[i]][7][:,:,::-1])

    ax[0,0].axis('off')
    ax[0,1].axis('off')
    ax[0,2].axis('off')
    ax[0,3].axis('off')
    ax[0,4].axis('off')
    ax[0,5].axis('off')
    ax[0,6].axis('off')
    ax[0,7].axis('off')
    ax[1,0].axis('off')
    ax[1,1].axis('off')
    ax[1,2].axis('off')
    ax[1,3].axis('off')
    ax[1,4].axis('off')
    ax[1,5].axis('off')
    ax[1,6].axis('off')
    ax[1,7].axis('off')

    plt.show()